In [13]:
import pyodbc
import pandas as pd
import numpy as np
import seaborn as sns
import urllib
import requests
import sqlalchemy as sqla
import matplotlib as mpl
import xml.etree.ElementTree as ET

from matplotlib import pyplot as plt
from importlib import reload
from tqdm import tqdm_notebook
from itertools import chain

In [14]:
with open('dbmi-aetna-cxn-str') as f:
    cxn_params = f.read().strip()
    gmw_cxn_str = urllib.parse.quote_plus(cxn_params + 'Database=gmw3')
    aetna_raw_cxn_str = urllib.parse.quote_plus(cxn_params + 'Database=AetnaDataWarehouse')
    hy_cxn_str = urllib.parse.quote_plus(cxn_params + 'Database=hy180')
    
engine = sqla.create_engine("mssql+pyodbc:///?odbc_connect=%s" % gmw_cxn_str, connect_args = {'autocommit':True})
cxn = engine.connect()
ins = sqla.inspect(engine)

In [15]:
hy_engine = sqla.create_engine("mssql+pyodbc:///?odbc_connect=%s" % hy_cxn_str, connect_args = {'autocommit': True, 'fast_executemany': True})
hy_cxn = hy_engine.connect()

# Enrollment min length

In [16]:
min_enrollment_query = ("drop table if exists hy180.dbo.EnrollFourPlusYears;"
                       " select MemberNum, LongestEnrollmentStartDate as enrollDate, LongestEnrollmentMonths as enrollTotalMonths, dateadd(m, 48, LongestEnrollmentStartDate) as enrollDatePlusFourYears"
                       " into hy180.dbo.EnrollFourPlusYears"
                       " from MemberEnrollment"
                       " where LongestEnrollmentMonths > 48")

cxn.execute(min_enrollment_query)
pd.read_sql('select count(*) from EnrollFourPlusYears', hy_cxn)

,
0,10546216


In [11]:
def default_enroll_window(select_cols,
                          claims_table, 
                          claim_date_col,
                          enrollment_table='hy180.dbo.EnrollFourPlusYears', 
                          window_date_col='enrollDatePlusFourYears',
                          enroll_date_col='enrollDate',
                          join_col='MemberNum'):
    
    select_cols.append(f't1.{join_col}')
    return (f"select {','.join(select_cols)}"
       f" from {claims_table} as t1 inner join {enrollment_table} as t2"
       f" on t1.{join_col}=t2.{join_col}"
       f" where {claim_date_col} between {enroll_date_col} and {window_date_col}")

In [7]:
def custom_enroll_window(select_cols, 
                          claims_table, 
                          claim_date_col,
                          claim_window=48,
                          window_unit='m',
                          enrollment_table='hy180.dbo.EnrollFourPlusYears', 
                          enrollment_date_col='enrollDate',
                          join_col='MemberNum'):
    
    select_cols.append(f't1.{join_col}')
    return (f"select {','.join(select_cols)}"
           f" from {claims_table} as t1 inner join {enrollment_table} as t2"
           f" on t1.{join_col}=t2.{join_col}"
           f" where datediff({window_unit}, {claim_date_col}, {enrollment_date_col}) <= {claim_window}")

# Helper tables for T2D 

## Visit count

In [67]:
# Visit count per patient

visit_count_query = ("drop table if exists hy180.dbo.VisitCount;"
                    " select MemberNum, count(*) as visits"
                    " into hy180.dbo.VisitCount"
                    f" from ({default_enroll_window([], 'VisitMedicalClaim', 'FirstServiceStartDate')}) as t"
                    " group by MemberNum")
cxn.execute(visit_count_query)
pd.read_sql('select count(*) from VisitCount', hy_cxn)

## Labs

In [8]:
# Patients with glucose/hba1c lab values

glucose_tests_loinc = ('1558-6', '2339-0', '2345-7')
a1c_tests_loinc = ('4548-4', '17856-6', '4549-2', '17855-8')
lab_tests_loinc = glucose_tests_loinc + a1c_tests_loinc

normal_labs_query = ("drop table if exists hy180.dbo.GlucoseLabs;"
                    " select distinct MemberNum, LoincCode, TestResultValue, ServiceStartDate as date"
                    " into hy180.dbo.GlucoseLabs"
                   f" from ({default_enroll_window(['LoincCode', 'ServiceStartDate', 'TestResultValue'], 'ObservationLab', 'ServiceStartDate')}) as t"
                    " where LoincCode in %s" % str(lab_tests_loinc))
cxn.execute(normal_labs_query)

pd.read_sql('select count(*) from GlucoseLabs', hy_cxn)

,
0,14078568


## Diagnosis

In [82]:
# Patients with diagnosis of diabetes

dm_dx_icd = str(('790.21', '790.22', '790.2', '790.29', '791.5', '277.7', 'V18.0', 'V77.1'))
dm_dx_icd_fuzzy = ' or '.join(["DiagnosisCode like '%s'" % s for s in ('250%', '648.8%', '648.0%')])

# ICD for Family hx included here 
diabetes_dx_query = ("drop table if exists hy180.dbo.DxDm;"
                    " select distinct MemberNum, DiagnosisCode as diagCode, StartDate as date"
                    " into hy180.dbo.DxDm"
                   f" from ({default_enroll_window(['DiagnosisCode', 'StartDate'], 'ObservationDiagnosis', 'StartDate')}) as t"
                    " where DiagnosisCode in %s"
                    " or %s" % (dm_dx_icd, dm_dx_icd_fuzzy))

cxn.execute(diabetes_dx_query)
pd.read_sql('select count(*) from DxDm', hy_cxn)

,
0,18882212


In [70]:
# Split into t1d and t2d tables
t1dm_icd_fuzzy = "diagCode like '250._1' or diagCode like '250._3'"
t2dm_icd_fuzzy = "diagCode like '250.[0,2-9]0' or diagCode like '250.[0,2-9]2'"

t1dm_query = ("drop table if exists hy180.dbo.DxT1dm;"
             " select MemberNum, diagCode, date"
             " into hy180.dbo.DxT1dm"
             " from hy180.dbo.DxDm with (nolock)"
             " where %s" % t1dm_icd_fuzzy)
t2dm_query = ("drop table if exists hy180.dbo.DxT2dm;"
             " select MemberNum, diagCode, date"
             " into hy180.dbo.DxT2dm"
             " from hy180.dbo.DxDm with (nolock)"
             " where %s;" % t2dm_icd_fuzzy)

cxn.execute(';'.join([t1dm_query, t2dm_query]))
print(pd.read_sql('select count(*) from DxT1dm', hy_cxn))
print(pd.read_sql('select count(*) from DxT2dm', hy_cxn))

          
0  2036761
           
0  19155565


## Meds

In [44]:
t1dm_rxnorm = (139825, 274783, 314684, 352385, 400008, 51428, 5856, 86009, 139953)
t2dm_rxnorm = (173, 10633, 2404, 4821, 217360, 4815, 25789, 73044, 274332, 6809, 84108, 33738, 72610, 16681, 30009, 593411, 60548)
dm_rxnorm = (126958, 412956, 412959, 637321, 668291, 668370, 686655, 692383, 748611, 880998, 881056, 751128, 847187, 847191, 847197, 847203, 847207, 847211, 847230, 847239, 847252, 847256, 847259, 847263, 847278, 847416, 847417, 806905, 806903, 408119)

### Ingredient labeling

In [23]:
all_drugs = pd.read_sql('select NationalDrugCode,NdcDescription,NumberOfClaims from ConceptMedication', cxn, index_col='NationalDrugCode')
all_drugs.to_csv('all_drugs.csv')

In [77]:
drug_ingredients_create_table = ("drop table if exists RxIngredients;"
                                " create table RxIngredients ("
                                "  ndc bigint not null,"
                                "  rxcui bigint,"
                                "  ingredientRxcui bigint"
                                " );"
                                " create index indexByNdc on RxIngredients (ndc);")
hy_cxn.execute(drug_ingredients_create_table)

In [78]:
# Ingredient fetching happens in an external script
drug_ingredients = pd.read_csv('drug_ingredients.csv', usecols=['ndc', 'rxcui', 'ingredient_rxcui'], dtype=np.int_)

rows = [str(tuple(l)) for l in drug_ingredients.values]
i = 0
for i in tqdm_notebook(range(len(rows)//1000 + 1)):
    insert_sql = ("insert into RxIngredients (ndc, rxcui, ingredientRxcui) values %s;" % ','.join(rows[1000*i:1000*(i+1)]))
    hy_cxn.execute(insert_sql)


HBox(children=(IntProgress(value=0, max=89), HTML(value='')))

### Insulin supplies

DM med supplies listed in emerge algorithm are not ingredient-level, but SCDs, so we need explicitly find related NDCs

In [ ]:
# Get NDCs relating to DM med supplies list
BASE_URL = "https://rxnav.nlm.nih.gov/REST"

explore_queue = list(dm_rxnorm)
all_rxcuis = set(dm_rxnorm)
insulin_ndcs = []

for rxcui in explore_queue:
    print(rxcui)
    # Get related RxCUIs 
    resp = requests.get(BASE_URL + '/rxcui/%s/allrelated' % rxcui)
    root = ET.fromstring(resp.text)
    related_rxcui = list(chain.from_iterable([[n.text for n in root.findall("./allRelatedGroup/conceptGroup/[tty='%s']/conceptProperties/rxcui" % tty)] for tty in ['SCD', 'GPCK', 'SBD', 'BPCK']]))
    explore_queue.extend(set(related_rxcui).difference(all_rxcuis))
    all_rxcuis = all_rxcuis.union(related_rxcui)
    
    # Get NDCs for RxCUI
    resp = requests.get(BASE_URL + '/rxcui/%s/allhistoricalndcs' % rxcui)
    root = ET.fromstring(resp.text)
    insulin_ndcs += [int(n.text) for n in root.findall("./historicalNdcConcept/historicalNdcTime/ndcTime/ndc")]
    
with open('insulin_ndcs', 'w') as f:
    for ndc in insulin_ndcs:
        f.write('%d\n' % ndc)

In [41]:
with open('insulin_ndcs') as f:
    insulin_ndcs = [int(l) for l in f.readlines()]

### Validation of ingredient labeling

In [79]:
# Claims coverage 
all_drugs.loc[drug_ingredients.ndc.unique()].NumberOfClaims.sum() / all_drugs.NumberOfClaims.sum()

0.9846076596542334

In [80]:
# Count DM-related drugs discovered
drug_ingredients[drug_ingredients.ingredient_rxcui.isin(t1dm_rxnorm) | drug_ingredients.ingredient_rxcui.isin(t2dm_rxnorm) | drug_ingredients.ingredient_rxcui.isin(dm_rxnorm)].shape[0]

2354

## Patients on DM meds

In [80]:
insulin_med_temp_query = ("drop table if exists #InsulinMeds;"
                          " select ingredientRxcui, ndc"
                          " into #InsulinMeds"
                          " from RxIngredients"
                          " where ndc in %s" % str(tuple(insulin_ndcs)))
insulin_rx_query = ("drop table if exists RxInsulin"
                    " select distinct t1.MemberNum, t2.ndc, t2.ingredientRxcui, t1.DispenseDate"
                    " into RxInsulin"
                   f" from ({default_enroll_window(['NationalDrugCode', 'DispenseDate'], 'gmw3.dbo.ObservationMedication', 'DispenseDate')}) as t1"
                    " inner join #InsulinMeds as t2"
                    " on t1.NationalDrugCode=t2.ndc")

hy_cxn.execute(';'.join([insulin_med_temp_query, insulin_rx_query]))
pd.read_sql('select count(*) from RxInsulin', hy_cxn)

,
0,642502


In [81]:
t1dm_med_temp_query = ("drop table if exists #T1DmMeds;"
                    " select ingredientRxcui, ndc"
                    " into #T1DmMeds"
                    " from RxIngredients"
                    " where ingredientRxcui in %s" % str(t1dm_rxnorm))
t1dm_rx_query = ("drop table if exists RxT1Dm"
              " select distinct t1.MemberNum, t2.ndc, t2.ingredientRxcui, t1.DispenseDate"
              " into RxT1Dm"
             f" from ({default_enroll_window(['NationalDrugCode', 'DispenseDate'], 'gmw3.dbo.ObservationMedication', 'DispenseDate')}) as t1"
              " inner join #T1DmMeds as t2"
              " on t1.NationalDrugCode=t2.ndc")

hy_cxn.execute(';'.join([t1dm_med_temp_query, t1dm_rx_query]))
pd.read_sql('select count(*) from RxT1Dm', hy_cxn)

,
0,1608401


In [79]:
t2dm_med_temp_query = ("drop table if exists #T2DmMeds;"
                    " select ingredientRxcui, ndc"
                    " into #T2DmMeds"
                    " from RxIngredients"
                    " where ingredientRxcui in %s" % str(t2dm_rxnorm))
t2dm_rx_query = ("drop table if exists RxT2dm;"
              " select distinct t1.MemberNum, t2.ndc, t2.ingredientRxcui, t1.DispenseDate"
              " into RxT2dm"
             f" from ({default_enroll_window(['NationalDrugCode', 'DispenseDate'], 'gmw3.dbo.ObservationMedication', 'DispenseDate')}) as t1"
              " inner join #T2DmMeds as t2"
              " on t1.NationalDrugCode=t2.ndc")

hy_cxn.execute(';'.join([t2dm_med_temp_query, t2dm_rx_query]))
pd.read_sql('select count(*) from RxT2dm', hy_cxn)

,
0,7460516


In [75]:
dm_rx_query = ("drop table if exists RxDm;"
              " select *"
              " into RxDm"
              " from (select * from RxT2dm union select * from RxInsulin union select * from RxT1dm) as t1")
hy_cxn.execute(dm_rx_query)
pd.read_sql('select count(*) from RxDm', hy_cxn)

,
0,9082635


# Helper tables for depression

In [20]:
depression_icds = ('293.83', '296.20', '296.21', '296.22', '296.23', '296.24', '296.25', '296.26', '296.30', '296.31', '296.32', '296.33', '296.34', '296.35', '296.36', '300.4', '311')

# Get gaps between depression claims 
depression_dx_query = ("drop table if exists hy180.dbo.dxDepression;"
                       " select distinct MemberNum, DiagnosisCode as diagCode, StartDate as date"
                       " into hy180.dbo.dxDepression"
                      f" from ({default_enroll_window(['DiagnosisCode', 'StartDate'], 'ObservationDiagnosis', 'StartDate')}) as t"
                      f" where DiagnosisCode in {str(depression_icds)}")

cxn.execute(depression_dx_query)
pd.read_sql('select count(*) from dxDepression', hy_cxn)

,
0,7734339
